## 데이터 로드

In [ ]:
import json
import pandas as pd
import math
import numpy as np

with open('../Datasets/train.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)

In [ ]:
train_data = pd.DataFrame(json_data)
train_data = train_data.drop(['id', 'plylst_title', 'updt_date'], axis=1)
train_data.head()

In [ ]:
with open('../Datasets/song_meta_with_likes.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)

In [ ]:
song_data = pd.DataFrame(json_data)
song_data = song_data.drop(['album_name', 'song_gn_gnr_basket'], axis=1)
song_data.head()

## 데이터 열 이름 변경

In [ ]:
train_data.rename(columns={'songs':'song_id'}, inplace=True)
train_data.head()

## 데이터 추출

- 500개의 플레이리스트 추출

In [ ]:
train_data_sample = train_data[500:1000].copy()
train_data_sample = train_data_sample.reset_index(drop=True)
#train_data_sample = train_data

In [ ]:
song_data.rename(columns={'id':'song_id', 'song_gn_dtl_gnr_basket': 'gnr'}, inplace=True)
song_data = song_data.astype({'issue_date':'int64'})
song_data.head()

## 노래별 가중치 설정

In [ ]:
# 좋아요 개수 분포 확인을 위한 코드
clean_song_data = song_data[song_data['like_cnt_song'] > 0]
clean_song_data['like_cnt_song'].hist(bins=100)
clean_song_data['like_cnt_song'].describe()

In [ ]:
bins = [-1, 0.001, 0.3, 1.5, 9, 3600]  # 구간 분할
labels = [0, 0.8, 0.85, 0.9, 0.95]  # 구간별 가중치 지정

song_data['weight'] = pd.cut(song_data['like_cnt_song'], bins=bins, labels=labels)  # 구간별 가중치 지정
song_data.head(10)

# 한국어 -> 영어 변환
- 한국어 전처리에 어려움이 있어서, 영어로 모두 변환 후 처리

In [ ]:
'''
from googletrans import Translator
import time

def trans_to_eng(tags):
    time.sleep(1)
    translator = Translator()
    translated = translator.translate(tags, src = 'ko', dest = 'en')
    return translated.text

for i in range(len(train_data_sample)):
    eng_tags = [trans_to_eng(tag) for tag in train_data_sample['tags'][i]]
    train_data_sample['tags'][i] = eng_tags

train_data_sample.to_json('../Datasets/train_eng1.json', orient='records')
train_data_sample.head(30)

    
#train_data.head()
#eng_tags = [trans_to_eng(tag) for tag in train_data['tags'][1]]
#print("Kor: ", train_data['tags'][1])
#print("Eng: ", eng_tags)
'''

In [ ]:
'''
with open('../Datasets/train_eng.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)
 '''

In [ ]:
'''
train_data_eng = pd.DataFrame(json_data)
train_data_eng.head(10)
'''

# 태그 자연어 전처리

### 특수문자 및 공백 제외 + 대->소문자 변환

In [ ]:
'''
import re

print(train_data_eng['tags'][3])
pattern = re.compile('[^a-zA-Z0-9]')#특수문자나 공백을 띄어쓰기로 처리할지는 이후 테스트
idx = 0
for tags in train_data_eng['tags']:
    eng_tags = []
    for tag in tags:
        temp_tags = re.sub(pattern, ' ', tag).lower().split()
        for temp_tag in temp_tags:
            eng_tags.append(temp_tag)
    train_data_eng['tags'][idx] = eng_tags
    idx += 1
print(train_data_eng['tags'][3])
'''

### Stopwords 제거

In [ ]:
'''
import nltk
from nltk.corpus import stopwords
#nltk.download('popular') # nltk 라이브러리 사용을 위해 다운해야 함

stops = set(stopwords.words('english'))

for i in range(len(train_data_eng)):
    eng_tags = [tag for tag in train_data_eng['tags'][i] if not tag in stops]
    if train_data_eng['tags'][i] != eng_tags:
        print(train_data_eng['tags'][i])
        print(eng_tags)
    train_data_eng['tags'][i] = eng_tags
    '''

### Stemming

In [ ]:
'''
stemmer = nltk.stem.SnowballStemmer('english')
for i in range(len(train_data_eng)):
    eng_tags = [stemmer.stem(tag) for tag in train_data_eng['tags'][i]]
    train_data_eng['tags'][i] = eng_tags
    '''

### 중복 제거

In [ ]:
'''
for i in range(len(train_data_eng)):
    eng_tags = list(dict.fromkeys(train_data_eng['tags'][i]))
    train_data_eng['tags'][i] = eng_tags
    '''

### 한 글자로 된 단어 제거

In [ ]:
'''
for i in range(len(train_data_eng)):
    eng_tags = [tag for tag in train_data_eng['tags'][i] if len(tag) > 1]
    train_data_eng['tags'][i] = eng_tags
    '''

### 추가적인 자연어 처리

In [ ]:
'''
# '록' 또는 '락' 이 'lock' 으로 번역되는 문제가 있어서, 'rock' 으로 일괄적으로 수정
pattern = re.compile(r'\block\b')
for i in range(len(train_data_eng)):
    eng_tags = [re.sub(pattern, 'rock', tag) for tag in train_data_eng['tags'][i]]
    train_data_eng['tags'][i] = eng_tags
    '''

In [ ]:
'''
train_data_sample = train_data_eng.copy()
train_data_sample = train_data_sample.reset_index(drop=True)
train_data_sample.head(20)
'''

## 태그 병합

- 같은 노래에 부여된 서로 다른 태그들을 합친다
- 그 결과 동일한 태그 리스트가 거의 모든 노래에 부여되었다

In [ ]:
train_data_sample = train_data_sample.explode('song_id', ignore_index=True)
train_data_sample.head(30)

In [ ]:
train_dict = dict()

for i in range(len(train_data_sample)):
    song = train_data_sample['song_id'][i]
    tag = train_data_sample['tags'][i]
    
    if song in train_dict:
        for j in tag:
            train_dict[song].add(j)
    
    else:
        train_dict[song] = set(tag)
        
print(train_dict[157435])

In [ ]:
train_data_sample.drop_duplicates(subset='song_id', keep='first',inplace=True)
train_data_sample.shape

In [ ]:
for i in range(len(train_data_sample)):
    song = train_data_sample['song_id'].iloc[i]
    
    train_data_sample['tags'].iloc[i] = list(train_dict[song])

train_data_sample.head()

In [ ]:
song_tag_appended = pd.merge(train_data_sample, song_data)
song_tag_appended = song_tag_appended.astype({'song_id':'int64'})
song_tag_appended.head()

In [ ]:
song_tag_appended.info()

## Word2Vec 사용

- 태그 리스트들을 word2vec로 학습시켜 태그 하나와 연관된 다른 태그들을 유추

In [ ]:
train_data_sample2 = train_data[500:1000].copy()
#train_data_sample2 = train_data_eng.copy()
train_data_sample2 = train_data_sample2.reset_index(drop=True)
#train_data_sample2 = train_data

In [ ]:
from gensim.models.word2vec import Word2Vec

w2v = Word2Vec(sentences = song_tag_appended['tags'], vector_size = 100, 
               window = 5, min_count = 15, workers = 4, sg = 1)

w2v.wv.vectors.shape

In [ ]:
#print(w2v.wv.most_similar('rock'))

# 태그 불균형

In [ ]:
def make_song_num_dict(data):
    song_ids = dict()
    song_num = dict()
    max_num = 0
    
    for i in range(len(data)):
        songs = data['song_id'][i]
        tags = data['tags'][i]
        
        for j in tags:
            if not j in song_ids:
                song_ids[j] = set(songs)
            
            else:
                song_ids[j].update(songs)
    
    for i in song_ids:
        song_num[i] = len(song_ids[i])
        
        max_num = max(song_num[i], max_num)
    
    return song_num, max_num

song_num_dict, song_num_max = make_song_num_dict(train_data_sample2)
tag_weights = {tag: np.log(song_num_max / cnt + 1) for tag, cnt in song_num_dict.items()}
print(tag_weights)


## 코사인 유사도 사용

- 세부 장르를 사용해 코사인 유사도 측정한다
- 그후 유사도를 행렬로 저장한다

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

song_tag_appended['gnr_literal'] = song_tag_appended['gnr'].apply(lambda x : (' ').join(x))

count_vect = CountVectorizer()
gnr_mat = count_vect.fit_transform(song_tag_appended['gnr_literal'])

gnr_mat.shape

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

gnr_sim = cosine_similarity(gnr_mat, gnr_mat)
gnr_sim

In [ ]:
test_data_sample = train_data[500:1000].copy()
#test_data_sample = train_data_eng.copy()
test_data_sample = test_data_sample.reset_index(drop=True)
test_my_tags = test_data_sample['tags'].tolist()
test_my_songs = test_data_sample['song_id'].tolist()

In [ ]:
from recommend import *

weight_mat_cv = apply_genre_weight(get_embedding(song_tag_appended, 'cv'))
weight_mat_tf = apply_genre_weight(get_embedding(song_tag_appended, 'tf'))

# 멀티프로세싱을 이용한 학습 및 결과 도출

In [ ]:
import multiprocessing

## (1) Rec1 & 코사인 유사도

In [ ]:
params_list = [
    #태그 / 장르 / 좋아요 / mode(cv/tf)
    (False, False, False, 'cv'),
    (True, False, False, 'cv'),
    (False, True, False, 'cv'),
    (True, True, False, 'cv'),
    (False, False, True, 'cv'),
    (True, False, True, 'cv'),
    (False, True, True, 'cv'),
    (True, True, True, 'cv'),
    #(False, False, False, 'tf'),
    #(True, False, False, 'tf'),
    #(False, True, False, 'tf'),
    #(True, True, False, 'tf'),
    #(False, False, True, 'tf'),
    #(True, False, True, 'tf'),
    #(False, True, True, 'tf'),
    #(True, True, True, 'tf')
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc_cos, [(test_my_songs, test_my_tags, song_tag_appended, param[0], param[1], param[2], 
                                         param[3], w2v, weight_mat_cv, weight_mat_tf, tag_weights) for param in params_list])
    
for result in results:
    print(result)

## (2) Rec1 & 피어슨 유사도

In [ ]:
params_list = [
    #태그 / 장르 / 좋아요 / mode(cv/tf)
    (False, False, False, 'cv'),
    (True, False, False, 'cv'),
    (False, True, False, 'cv'),
    (True, True, False, 'cv'),
    (False, False, True, 'cv'),
    (True, False, True, 'cv'),
    (False, True, True, 'cv'),
    (True, True, True, 'cv'),
    #(False, False, False, 'tf'),
    #(True, False, False, 'tf'),
    #(False, True, False, 'tf'),
    #(True, True, False, 'tf'),
    #(False, False, True, 'tf'),
    #(True, False, True, 'tf'),
    #(False, True, True, 'tf'),
    #(True, True, True, 'tf')
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc_pea, [(test_my_songs, test_my_tags, song_tag_appended, param[0], param[1], param[2], 
                                         param[3], w2v, weight_mat_cv, weight_mat_tf, tag_weights) for param in params_list])
    
for result in results:
    print(result)

## (3) Rec2 & 코사인 유사도

In [ ]:
params_list = [
    #태그 / 장르 / 좋아요 / mode(cv/tf)
    (False, False, False, 'cv'),
    (True, False, False, 'cv'),
    (False, True, False, 'cv'),
    (True, True, False, 'cv'),
    (False, False, True, 'cv'),
    (True, False, True, 'cv'),
    (False, True, True, 'cv'),
    (True, True, True, 'cv'),
    #(False, False, False, 'tf'),
    #(True, False, False, 'tf'),
    #(False, True, False, 'tf'),
    #(True, True, False, 'tf'),
    #(False, False, True, 'tf'),
    #(True, False, True, 'tf'),
    #(False, True, True, 'tf'),
    #(True, True, True, 'tf')
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc_cos2, [(test_my_songs, test_my_tags, song_tag_appended, param[0], param[1], param[2], 
                                         param[3], w2v, weight_mat_cv, weight_mat_tf, tag_weights) for param in params_list])
    
for result in results:
    print(result)

## (4) Rec2 & 피어슨 유사도

In [ ]:
params_list = [
    #태그 / 장르 / 좋아요 / mode(cv/tf)
    (False, False, False, 'cv'),
    (True, False, False, 'cv'),
    (False, True, False, 'cv'),
    (True, True, False, 'cv'),
    (False, False, True, 'cv'),
    (True, False, True, 'cv'),
    (False, True, True, 'cv'),
    (True, True, True, 'cv'),
    #(False, False, False, 'tf'),
    #(True, False, False, 'tf'),
    #(False, True, False, 'tf'),
    #(True, True, False, 'tf'),
    #(False, False, True, 'tf'),
    #(True, False, True, 'tf'),
    #(False, True, True, 'tf'),
    #(True, True, True, 'tf')
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc_pea2, [(test_my_songs, test_my_tags, song_tag_appended, param[0], param[1], param[2], 
                                         param[3], w2v, weight_mat_cv, weight_mat_tf, tag_weights) for param in params_list])
    
for result in results:
    print(result)

In [ ]:
rec1 = song_recommend(test_my_tags[0], test_my_songs[0], song_tag_appended, 'cos', False, False, False, 'cv')

pred_list = rec1['song_id'].tolist()
precision_k = get_precision_k(test_my_songs[0], pred_list)
recall_k = get_recall_k(test_my_songs[0], pred_list)
ndcg = get_ndcg(test_my_songs[0], pred_list, 10)

print("Recall@K (K=10): {:.2f}".format(recall_k))
print("Precision@K (K=10): {:.2f}".format(precision_k))
print("nDCG: {:.2f}".format(ndcg))